In [1]:
!pip install flask flask-cors pyngrok tensorflow pillow


In [2]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import tensorflow as tf
import numpy as np
from PIL import Image
import io
from pyngrok import ngrok
from threading import Thread

# Create Flask app
app = Flask(__name__)
CORS(app)

# Load your trained model
model = tf.keras.models.load_model("waste_classifier.keras")

# Classes
class_names = ["battery", "biological", "brown-glass", "cardboard", "clothes", "green-glass", "metal", "paper", "plastic", "shoes", "trash", "white-glass"]


# Bin suggestions for each class
bin_suggestions = {
    "battery": {
        "bin": "Hazardous Waste Bin",
        "suggestion": "Do not throw batteries in regular trash. Dispose at a battery recycling center or e-waste bin."
    },
    "biological": {
        "bin": "Compost Bin",
        "suggestion": "Biodegradable waste should go into compost — like food scraps or garden waste."
    },
    "brown-glass": {
        "bin": "Recycling Bin (Glass)",
        "suggestion": "Clean and place in the brown-glass recycling bin."
    },
    "cardboard": {
        "bin": "Recycling Bin (Paper/Cardboard)",
        "suggestion": "Flatten boxes and place in the paper/cardboard recycling bin."
    },
    "clothes": {
        "bin": "Donation or Textile Recycling",
        "suggestion": "Consider donating or putting in textile recycling collection bins."
    },
    "green-glass": {
        "bin": "Recycling Bin (Glass)",
        "suggestion": "Clean and place in the green-glass recycling bin."
    },
    "metal": {
        "bin": "Recycling Bin (Metal)",
        "suggestion": "Rinse and recycle — avoid putting sharp metal objects in the bin."
    },
    "paper": {
        "bin": "Recycling Bin (Paper)",
        "suggestion": "Recycle clean paper; avoid soiled or wet paper."
    },
    "plastic": {
        "bin": "Recycling Bin (Plastic)",
        "suggestion": "Check local recycling rules; rinse containers before recycling."
    },
    "shoes": {
        "bin": "Donation or Textile Recycling",
        "suggestion": "If reusable, donate; otherwise, recycle through textile programs."
    },
    "trash": {
        "bin": "General Waste Bin",
        "suggestion": "Dispose of non-recyclable items in the general waste bin."
    },
    "white-glass": {
        "bin": "Recycling Bin (Glass)",
        "suggestion": "Clean and place in the white-glass recycling bin."
    }
}


# Test route
@app.route('/')
def home():
    return "✅ Waste Classifier API is running!"

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    file = request.files['file']
    img = Image.open(io.BytesIO(file.read())).convert('RGB')
    img = img.resize((224, 224))
    img_array = np.expand_dims(np.array(img)/255.0, axis=0)

    preds = model.predict(img_array)
    class_idx = int(np.argmax(preds[0]))
    confidence = float(np.max(preds[0]))
    predicted_class = class_names[class_idx].strip().lower()  # normalize

    # Get bin and suggestion info
    info = bin_suggestions.get(predicted_class, {"bin": "Unknown", "suggestion": "No suggestion available."})

    print(f"Predicted class: {predicted_class}, Bin: {info['bin']}, Suggestion: {info['suggestion']}")

    return jsonify({
        'predicted_class': predicted_class,
        'confidence': confidence,
        'bin': info['bin'],
        'suggestion': info['suggestion']
    })


# Function to run Flask in background
def run():
    app.run(port=5000)

# Start Flask
Thread(target=run).start()


In [3]:
!ngrok authtoken 347CnKnBIAtcJ4q8P5lh25ssb1h_5D8VbG5BivuFKSVKYrGPY


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
# Expose the API publicly
public_url = ngrok.connect(5000)
print("🚀 Public URL:", public_url)


🚀 Public URL: NgrokTunnel: "https://hypernutritive-marley-untheoretically.ngrok-free.dev" -> "http://localhost:5000"
